In [15]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import randint
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.svm import SVC
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import scipy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE


In [16]:
#Descarga de los datos
DataIMCO_N = pd.read_csv("IMCO_N.csv",low_memory=False)
DataIMCO_PR = pd.read_csv("IMCO_PreRed.csv",low_memory=False)


In [17]:
print(DataIMCO_N.shape)
print(DataIMCO_PR.shape)

(667316, 47)
(667316, 61)


In [20]:

DataIMCO_N = DataIMCO_N.drop(['Unnamed: 0'], axis=1)
DataIMCO_PR = DataIMCO_PR.drop(['Unnamed: 0'], axis=1)


X_n = DataIMCO_N.drop('Status',axis=1)
y_n = DataIMCO_N['Status']

X_trainn, X_testn, y_trainn, y_testn = train_test_split(X_n, y_n, test_size=0.3)

smote = SMOTE(random_state=17)

X_trainn, y_trainn = smote.fit_resample(X_trainn, y_trainn)
#X_testn, y_testn = smote.fit_resample(X_testn,y_testn)


X_r = DataIMCO_PR.drop('Status',axis=1)
y_r = DataIMCO_PR['Status']

X_trainp, X_testp, y_trainp, y_testp = train_test_split(X_r, y_r, test_size=0.3)

smote = SMOTE(random_state=17)

X_trainp, y_trainp = smote.fit_resample(X_trainp, y_trainp)
#X_testp, y_testp = smote.fit_resample(X_testp,y_testp)

In [21]:
print(DataIMCO_N.shape)
print(DataIMCO_PR.shape)

(667316, 46)
(667316, 60)


In [22]:
print(y_trainp.shape)
print(y_testp.shape)

(932624,)
(200195,)


In [23]:
print(y_trainn.shape)
print(y_testn.shape)

(932614,)
(200195,)


In [24]:
print(y_trainn.value_counts())
print(y_testn.value_counts())

0    466307
1    466307
Name: Status, dtype: int64
0    199882
1       313
Name: Status, dtype: int64


In [25]:
print(y_trainp.value_counts())
print(y_testp.value_counts())

0    466312
1    466312
Name: Status, dtype: int64
0    199877
1       318
Name: Status, dtype: int64


# Paqueterias

## Random Forest

### IMCO

In [26]:
{'n_estimators': 4000, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 80, 'bootstrap': False}
IMCO_rf_norm = RandomForestClassifier(n_estimators=4000,
                                      min_samples_split=10,
                                      min_samples_leaf=2,
                                      max_features='sqrt',
                                      max_depth=80,
                                      bootstrap=False)
IMCO_rf_norm.fit(X_trainn,y_trainn)
IMCOy_pred_rand_norm = IMCO_rf_norm.predict(X_testn)

print('RANDOM FOREST NORMAL DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testn,IMCOy_pred_rand_norm))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testn, IMCOy_pred_rand_norm))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testn, IMCOy_pred_rand_norm))


RANDOM FOREST NORMAL DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199882
           1       0.13      0.09      0.11       313

    accuracy                           1.00    200195
   macro avg       0.56      0.54      0.55    200195
weighted avg       1.00      1.00      1.00    200195

CONFUSION MATRIX
[[199691    191]
 [   285     28]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.5442506526134765


In [ ]:
feat_importances_norm = pd.Series(IMCO_rf_norm.feature_importances_, index=X_trainn.columns)
feat_importances_norm.nlargest(10).plot(kind='barh')

In [27]:
# {'n_estimators': 2500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}
IMCO_rf_red = RandomForestClassifier(n_estimators=4000,
                                      min_samples_split=5,
                                      min_samples_leaf=2,
                                      max_features='sqrt',
                                      max_depth=50,
                                      bootstrap=False)
IMCO_rf_red.fit(X_trainp,y_trainp)
IMCOy_pred_rand_red = IMCO_rf_red.predict(X_testp)


print('RANDOM FOREST RED FLAG DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testp,IMCOy_pred_rand_red))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testp, IMCOy_pred_rand_red))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testp, IMCOy_pred_rand_red))



RANDOM FOREST RED FLAG DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199877
           1       1.00      0.70      0.82       318

    accuracy                           1.00    200195
   macro avg       1.00      0.85      0.91    200195
weighted avg       1.00      1.00      1.00    200195

CONFUSION MATRIX
[[199877      0]
 [    96    222]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.8490566037735849


In [ ]:

feat_importances_red = pd.Series(IMCO_rf_red.feature_importances_, index=X_trainp.columns)
feat_importances_red.nlargest(10).plot(kind='barh')

# XGBOOSTCLASSIFIER

In [37]:
import sys

!{sys.executable} -m pip install xgboost
from xgboost import XGBClassifier

In [38]:
# {'subsample': 0.9, 'n_estimators': 300, 'max_depth': 11, 'learning_rate': 0.1}

IMCO_XGBC_norm = XGBClassifier(subsample=0.9,
                          n_estimators=300,
                          max_depth=11,
                          learning_rate=0.1)

IMCO_XGBC_norm.fit(X_trainn, y_trainn)

IMCO_pred_xgbc_norm = IMCO_XGBC_norm.predict(X_testn)

print('XGBOOST CLASSIFIER NORMAL DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testn, IMCO_pred_xgbc_norm))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testn, IMCO_pred_xgbc_norm))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testn, IMCO_pred_xgbc_norm))

XGBOOST CLASSIFIER NORMAL DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199882
           1       0.11      0.05      0.07       313

    accuracy                           1.00    200195
   macro avg       0.55      0.52      0.53    200195
weighted avg       1.00      1.00      1.00    200195

CONFUSION MATRIX
[[199762    120]
 [   298     15]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.5236614842373615


In [39]:
# {'subsample': 0.8, 'n_estimators': 150, 'max_depth': 11, 'learning_rate': 0.1}
IMCO_XGBC_red = XGBClassifier(subsample=0.8,
                          n_estimators=300,
                          max_depth=11,
                          learning_rate=0.1)


IMCO_XGBC_red.fit(X_trainp, y_trainp)

IMCO_pred_xgbc_red = IMCO_XGBC_red.predict(X_testp)

print('XGBOOST CLASSIFIER PREPROCESSED & RED DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testp, IMCO_pred_xgbc_red))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testp, IMCO_pred_xgbc_red))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testp, IMCO_pred_xgbc_red))


XGBOOST CLASSIFIER PREPROCESSED & RED DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199877
           1       1.00      0.86      0.92       318

    accuracy                           1.00    200195
   macro avg       1.00      0.93      0.96    200195
weighted avg       1.00      1.00      1.00    200195

CONFUSION MATRIX
[[199876      1]
 [    44    274]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.9308151085244469


#  EXTRATREESCLASSIFIER

In [40]:

# {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None}
IMCO_ETC_norm = ExtraTreesClassifier(n_estimators=500,
                                     min_samples_split=2,
                                     min_samples_leaf=1,
                                     max_depth=None)

IMCO_ETC_norm.fit(X_trainn, y_trainn)

IMCO_pred_etc_norm = IMCO_ETC_norm.predict(X_testn)

print('EXTRATREESCLASSIFIER NORMAL DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testn, IMCO_pred_etc_norm))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testn, IMCO_pred_etc_norm))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testn, IMCO_pred_etc_norm))

EXTRATREESCLASSIFIER NORMAL DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199882
           1       0.08      0.09      0.09       313

    accuracy                           1.00    200195
   macro avg       0.54      0.55      0.54    200195
weighted avg       1.00      1.00      1.00    200195

CONFUSION MATRIX
[[199545    337]
 [   284     29]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.5454828812258018


In [41]:
# {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': None}

IMCO_ETC_red = ExtraTreesClassifier(n_estimators=500,
                                     min_samples_split=10,
                                     min_samples_leaf=1,
                                     max_depth=None)

IMCO_ETC_red.fit(X_trainp, y_trainp)

IMCO_pred_etc_red = IMCO_ETC_red.predict(X_testp)

print('EXTRATREESCLASSIFIER REDFLAGS DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testp, IMCO_pred_etc_red))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testp, IMCO_pred_etc_red))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testp, IMCO_pred_etc_red))

EXTRATREESCLASSIFIER REDFLAGS DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    199877
           1       1.00      0.64      0.78       318

    accuracy                           1.00    200195
   macro avg       1.00      0.82      0.89    200195
weighted avg       1.00      1.00      1.00    200195

CONFUSION MATRIX
[[199877      0]
 [   116    202]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.8176100628930818


# MULTILAYER PERCEPTRON

In [42]:
# {'solver': 'adam', 'max_iter': 300, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (100,), 'alpha': 0.001, 'activation': 'relu'}
IMCO_MLP_norm = MLPClassifier(solver='adam',
                    max_iter=300,
                    learning_rate='adaptive',
                    hidden_layer_sizes=(100,),
                    alpha=0.001,
                    activation='relu')

# Perform randomized search with cross-validation

IMCO_MLP_norm.fit(X_trainn, y_trainn)

IMCO_pred_mlpc_norm = IMCO_MLP_norm.predict(X_testn)

print('MLP NORMAL DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testn, IMCO_pred_mlpc_norm))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testn, IMCO_pred_mlpc_norm))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testn, IMCO_pred_mlpc_norm))

MLP NORMAL DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.79      0.88    199882
           1       0.00      0.61      0.01       313

    accuracy                           0.79    200195
   macro avg       0.50      0.70      0.44    200195
weighted avg       1.00      0.79      0.88    200195

CONFUSION MATRIX
[[157092  42790]
 [   122    191]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.6980736685762811


In [43]:
# {'solver': 'adam', 'max_iter': 100, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (100, 50), 'alpha': 0.001, 'activation': 'logistic'}

IMCO_MLP_red = MLPClassifier(solver='adam',
                    max_iter=300,
                    learning_rate='adaptive',
                    hidden_layer_sizes=(100,50),
                    alpha=0.001,
                    activation='logistic')

# Perform randomized search with cross-validation

IMCO_MLP_red.fit(X_trainp, y_trainp)

IMCO_pred_mlpc_red = IMCO_MLP_red.predict(X_testp)

print('MLP REDFLAGS DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testp, IMCO_pred_mlpc_red))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testp, IMCO_pred_mlpc_red))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testp, IMCO_pred_mlpc_red))


MLP REDFLAGS DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.27      0.43    199877
           1       0.00      0.87      0.00       318

    accuracy                           0.27    200195
   macro avg       0.50      0.57      0.22    200195
weighted avg       1.00      0.27      0.43    200195

CONFUSION MATRIX
[[ 54617 145260]
 [    40    278]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.5737334435520612


# NAIVE BAYES

In [44]:
# 'var_smoothing': 8.111308307896872e-05
IMCO_NBclass_norm = GaussianNB(var_smoothing=8.111308307896872e-05)

IMCO_NBclass_norm.fit(X_trainn, y_trainn)

IMCO_pred_nbclass_norm = IMCO_NBclass_norm.predict(X_testn)

print('NAIVE BAYES CLASSIFIER NORMAL DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testn, IMCO_pred_nbclass_norm))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testn, IMCO_pred_nbclass_norm))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testn, IMCO_pred_nbclass_norm))

NAIVE BAYES CLASSIFIER NORMAL DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.01      0.02    199882
           1       0.00      0.98      0.00       313

    accuracy                           0.01    200195
   macro avg       0.50      0.50      0.01    200195
weighted avg       1.00      0.01      0.02    200195

CONFUSION MATRIX
[[  2398 197484]
 [     6    307]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.4964138746013502


In [45]:
# 'var_smoothing': 8.111308307896872e-05
IMCO_NBclass_red = GaussianNB(var_smoothing=2.848035868435799e-08)

IMCO_NBclass_red.fit(X_trainp, y_trainp)

IMCO_pred_nbclass_red = IMCO_NBclass_red.predict(X_testp)

print('NAIVE BAYES CLASSIFIER NORMAL DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testp, IMCO_pred_nbclass_red))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testp, IMCO_pred_nbclass_red))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testp, IMCO_pred_nbclass_red))

NAIVE BAYES CLASSIFIER NORMAL DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.10      0.18    199877
           1       0.00      0.97      0.00       318

    accuracy                           0.10    200195
   macro avg       0.50      0.54      0.09    200195
weighted avg       1.00      0.10      0.18    200195

CONFUSION MATRIX
[[ 20219 179658]
 [     8    310]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.537999989490392


# KNeighborsClassifier

In [47]:
# {'weights': 'uniform', 'n_neighbors': 1, 'metric': 'euclidean'}
IMCO_knc_norm = KNeighborsClassifier(weights='uniform',
                                n_neighbors=1,
                                metric='euclidean')
IMCO_knc_norm.fit(X_trainn, y_trainn)

IMCO_pred_knc_norm = IMCO_knc_norm.predict(X_testn)

print('NEAREST NEIGHBOR CLASSIFIER NORMAL DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testn, IMCO_pred_knc_norm))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testn, IMCO_pred_knc_norm))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testn, IMCO_pred_knc_norm))

NEAREST NEIGHBOR CLASSIFIER NORMAL DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.83      0.91    199882
           1       0.00      0.34      0.01       313

    accuracy                           0.83    200195
   macro avg       0.50      0.59      0.46    200195
weighted avg       1.00      0.83      0.91    200195

CONFUSION MATRIX
[[166706  33176]
 [   206    107]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.5879375540834268


In [48]:
# {'weights': 'uniform', 'n_neighbors': 1, 'metric': 'euclidean'}
IMCO_knc_red = KNeighborsClassifier(weights='uniform',
                                n_neighbors=1,
                                metric='euclidean')
IMCO_knc_red.fit(X_trainp, y_trainp)

IMCO_pred_knc_red = IMCO_knc_red.predict(X_testp)

print('NEAREST NEIGHBOR CLASSIFIER NORMAL DATA')
print('TEST PERFORMANCE')
print('------------------------------------------------------------')
print(classification_report(y_testp, IMCO_pred_knc_red))
print('CONFUSION MATRIX')
print(confusion_matrix(y_testp, IMCO_pred_knc_red))
print('------------------------------------------------------------')
print('ROC_AUC_SCORE')
print('------------------------------------------------------------')
print(roc_auc_score(y_testp, IMCO_pred_knc_red))

NEAREST NEIGHBOR CLASSIFIER NORMAL DATA
TEST PERFORMANCE
------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    199877
           1       0.15      0.81      0.25       318

    accuracy                           0.99    200195
   macro avg       0.57      0.90      0.62    200195
weighted avg       1.00      0.99      0.99    200195

CONFUSION MATRIX
[[198416   1461]
 [    61    257]]
------------------------------------------------------------
ROC_AUC_SCORE
------------------------------------------------------------
0.9004333026446486


# LOGISTIC REGRESSION

In [ ]:
IMVO_LR_norm = LogisticRegression()


IMVO_LR_norm.fit(X_trainn, y_trainn)

y_pred_lr_norm = IMVO_LR_norm.predict(X_testn)

print(classification_report(y_pred_lr_norm,y_testn))
